# Load libraries

In [2]:
# load pandas and numpy
import pandas as pd
import numpy as np
import re
import os
import sys

# Read and create table for no-covariates models

In [2]:
models = ['arima', 'nhits', 'tft', 'linreg', 'xgboost', 'transformer']
datasets = ['weinstock', 'dubosson', 'colas', 'iglu', 'hall']
time_steps = 12

# create dataframes for MSE and MAE with columns for each dataset and rows for each model
df_mse = pd.DataFrame(columns=[dataset + ' ID' for dataset in datasets] + 
                      [dataset + ' OOD' for dataset in datasets], index=models)
df_mae = pd.DataFrame(columns=[dataset + ' ID' for dataset in datasets] +
                        [dataset + ' OOD' for dataset in datasets], index=models)
df_likelihood = pd.DataFrame(columns=[dataset + ' ID' for dataset in datasets] +
                        [dataset + ' OOD' for dataset in datasets], index=models[1:]) # no likelihood for ARIMA
df_calibration = pd.DataFrame(columns=[dataset + ' ID' for dataset in datasets] +
                        [dataset + ' OOD' for dataset in datasets], index=models[1:]) # no calibration for ARIMA
for model in models:
    for dataset in datasets:
        filename = f"./output/{model}_{dataset}.txt"
        if not os.path.isfile(filename):
            continue
        with open(filename, 'r') as f:
            for line in f:
                if line.startswith('ID median of (MSE, MAE):'):
                    id_mse_mae = re.findall(r'\d+\.\d+', line)
                    df_mse.loc[model, dataset + ' ID'] = float(id_mse_mae[0])
                    df_mae.loc[model, dataset + ' ID'] = float(id_mse_mae[1])
                elif line.startswith('OOD median of (MSE, MAE):'):
                    ood_mse_mae = re.findall(r'\d+\.\d+', line)
                    df_mse.loc[model, dataset + ' OOD'] = float(ood_mse_mae[0])
                    df_mae.loc[model, dataset + ' OOD'] = float(ood_mse_mae[1])
                elif line.startswith('ID likelihoods:'):
                    id_likelihoods = re.findall(r'-?\d+\.\d+', line)
                    df_likelihood.loc[model, dataset + ' ID'] = float(id_likelihoods[0])
                elif line.startswith('OOD likelihoods:'):
                    ood_likelihoods = re.findall(r'-?\d+\.\d+', line)
                    df_likelihood.loc[model, dataset + ' OOD'] = float(ood_likelihoods[0])
                elif line.startswith('ID calibration errors:'):
                    id_calib = re.findall(r'-?\d+\.\d+', line)
                    id_calib = np.mean([float(x) for x in id_calib[:time_steps]])
                    df_calibration.loc[model, dataset + ' ID'] = id_calib
                elif line.startswith('OOD calibration errors:'):
                    ood_calib = re.findall(r'-?\d+\.\d+', line)
                    ood_calib = np.mean([float(x) for x in ood_calib[:time_steps]])
                    df_calibration.loc[model, dataset + ' OOD'] = ood_calib
# sort columns for all dataframes
df_mse = df_mse.reindex(sorted(df_mse.columns), axis=1)
df_mae = df_mae.reindex(sorted(df_mae.columns), axis=1)
df_likelihood = df_likelihood.reindex(sorted(df_likelihood.columns), axis=1)
df_calibration = df_calibration.reindex(sorted(df_calibration.columns), axis=1)


In [4]:
df_mse

,colas ID,colas OOD,dubosson ID,dubosson OOD,hall ID,hall OOD,iglu ID,iglu OOD,weinstock ID,weinstock OOD
arima,33.59905,34.96897,183.109673,351.526345,74.521695,67.553378,110.89656,138.08389,179.540165,251.968444
nhits,35.203373,32.223549,304.597168,331.178528,59.021507,59.968391,190.204453,214.264008,176.602966,210.942352
tft,31.581922,30.395976,337.308976,306.185947,62.664102,65.953053,188.639482,154.458145,NaN,NaN
linreg,27.704992,28.412064,145.646805,246.192169,54.511375,61.780041,136.491379,143.425507,185.039337,242.589111
xgboost,41.234928,38.189861,448.432709,308.871643,57.448833,56.149467,163.829254,95.324974,185.873749,236.054855
transformer,41.916981,35.646248,276.329376,246.659943,62.220619,66.892151,228.614655,197.031128,174.869751,200.225235


# Read and create table for with-covariates models

In [3]:
models = ['nhits', 'tft', 'linreg', 'xgboost', 'transformer']
datasets = ['weinstock', 'dubosson', 'colas', 'iglu', 'hall']
time_steps = 12

# create dataframes for MSE and MAE with columns for each dataset and rows for each model
df_mse = pd.DataFrame(columns=[dataset + ' ID' for dataset in datasets] + 
                      [dataset + ' OOD' for dataset in datasets], index=models)
df_mae = pd.DataFrame(columns=[dataset + ' ID' for dataset in datasets] +
                        [dataset + ' OOD' for dataset in datasets], index=models)
df_likelihood = pd.DataFrame(columns=[dataset + ' ID' for dataset in datasets] +
                        [dataset + ' OOD' for dataset in datasets], index=models)
df_calibration = pd.DataFrame(columns=[dataset + ' ID' for dataset in datasets] +
                        [dataset + ' OOD' for dataset in datasets], index=models)
for model in models:
    for dataset in datasets:
        filename = f"./output/{model}_covariates_{dataset}.txt"
        if not os.path.isfile(filename):
            continue
        with open(filename, 'r') as f:
            for line in f:
                if line.startswith('ID median of (MSE, MAE):'):
                    id_mse_mae = re.findall(r'\d+\.\d+', line)
                    df_mse.loc[model, dataset + ' ID'] = float(id_mse_mae[0])
                    df_mae.loc[model, dataset + ' ID'] = float(id_mse_mae[1])
                elif line.startswith('OOD median of (MSE, MAE):'):
                    ood_mse_mae = re.findall(r'\d+\.\d+', line)
                    df_mse.loc[model, dataset + ' OOD'] = float(ood_mse_mae[0])
                    df_mae.loc[model, dataset + ' OOD'] = float(ood_mse_mae[1])
                elif line.startswith('ID likelihoods:'):
                    id_likelihoods = re.findall(r'-?\d+\.\d+', line)
                    df_likelihood.loc[model, dataset + ' ID'] = float(id_likelihoods[0])
                elif line.startswith('OOD likelihoods:'):
                    ood_likelihoods = re.findall(r'-?\d+\.\d+', line)
                    df_likelihood.loc[model, dataset + ' OOD'] = float(ood_likelihoods[0])
                elif line.startswith('ID calibration errors:'):
                    id_calib = re.findall(r'-?\d+\.\d+', line)
                    id_calib = np.mean([float(x) for x in id_calib[:time_steps]])
                    df_calibration.loc[model, dataset + ' ID'] = id_calib
                elif line.startswith('OOD calibration errors:'):
                    ood_calib = re.findall(r'-?\d+\.\d+', line)
                    ood_calib = np.mean([float(x) for x in ood_calib[:time_steps]])
                    df_calibration.loc[model, dataset + ' OOD'] = ood_calib
# sort columns for all dataframes
df_mse = df_mse.reindex(sorted(df_mse.columns), axis=1)
df_mae = df_mae.reindex(sorted(df_mae.columns), axis=1)
df_likelihood = df_likelihood.reindex(sorted(df_likelihood.columns), axis=1)
df_calibration = df_calibration.reindex(sorted(df_calibration.columns), axis=1)

In [7]:
df_calibration

,colas ID,colas OOD,dubosson ID,dubosson OOD,hall ID,hall OOD,iglu ID,iglu OOD,weinstock ID,weinstock OOD
nhits,0.214447,0.141414,0.080689,0.197497,0.06681,0.064741,0.071817,0.063145,0.107733,0.112227
tft,0.160679,0.086539,0.249412,0.352475,0.075971,0.083475,0.302196,0.233199,NaN,NaN
linreg,0.186624,0.170056,0.205757,0.47961,0.191934,0.234557,0.131142,0.551739,0.112624,0.207536
xgboost,0.088209,0.084381,0.064593,0.071753,0.092588,0.097941,0.066762,0.113061,0.102187,0.104396
transformer,0.306291,0.219041,0.178524,0.246606,0.095531,0.081821,0.212265,0.110486,0.107479,0.120045


# Clearing the results

In [1]:
import os

# Set the directory path to the folder containing the output files
folder_path = './output'

# Loop through each file in the folder
for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)
    # Open the file in read mode if the file starts with transformer
    if filename.startswith('transformer') or \
            filename.startswith('tft') or \
                filename.startswith('linreg') or \
                    filename.startswith('xgboost') or \
                        filename.startswith('nhits'):
        with open(file_path, 'r') as f:
            lines = f.readlines()
            
        # Loop through the lines in reverse order
        for i in range(len(lines)-1, -1, -1):
            if lines[i].startswith('Best value: '):
                # Delete all lines after the line starting with "Best value: "
                del lines[i+1:]
                break
        
        # Open the file in write mode and write the modified lines back to the file
        with open(file_path, 'w') as f:
            f.writelines(lines)
